# Train and Predict with pipelines

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv('/kaggle/input/titanic/titanicdata.csv')

In [5]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
df.drop(columns= ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace= True)

In [7]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
X_train, X_test, y_train, y_test= train_test_split(df.drop(columns= 'Survived'),
                                                   df['Survived'], 
                                                   test_size= 0.2, 
                                                   random_state= 42)

In [9]:
X_train.shape

(712, 7)

In [10]:
y_train

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

In [11]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [12]:
#missing value
missing_value= ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')


In [13]:
#One Hot Encoding

categorical_value= ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [14]:
#scaling

scaler_value= ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [15]:
#feature selection
fe_selection= SelectKBest(score_func=chi2,k=8) #top 8 columns selected

In [16]:
#train the model
d_tree= DecisionTreeClassifier()

# Create Pipelines

In [17]:
pipe= Pipeline([
    ('missing_value',missing_value),
    ('categorical_value',categorical_value),
    ('scaler_value',scaler_value),
    ('fe_selection',fe_selection),
    ('d_tree', d_tree)
])

# Pipeline Vs make_pipeline

pipeline requires naming of steps, make_pipeline dose not.
(same applies to ColumnTransformer vs make_column_transformer)

In [18]:
#alternate syntax
# pipe= make_pipeline(missing_value, categorical_value, scaler_value, fe_selection, d_tree)

In [19]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('missing_value',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('categorical_value',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('scaler_value',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('fe_selection',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7c6455e27880>)),
                ('d_tree', DecisionTreeClassifier())])

# Learn pipeline

In [20]:
pipe.named_steps

{'missing_value': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'categorical_value': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'scaler_value': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'fe_selection': SelectKBest(k=8, score_func=<function chi2 at 0x7c6455e27880>),
 'd_tree': DecisionTreeClassifier()}

In [21]:
pipe.named_steps['missing_value']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [22]:
pipe.named_steps['missing_value'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [23]:
pipe.named_steps['missing_value'].transformers_[0][1].statistics_

array([29.49884615])

In [24]:
pipe.named_steps['missing_value'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [25]:
#display pipeline
from sklearn import set_config
set_config(display= 'diagram')

In [26]:
#predict
y_pred= pipe.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6256983240223464

# Cross validation using pipeline

In [28]:
#cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv= 5, scoring= 'accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [29]:
#gridsearchcv
params= {
    'd_tree__max_depth': [1, 2, 3, 4, 5, None]
}

In [30]:
from sklearn.model_selection import GridSearchCV
grid= GridSearchCV(pipe, params, cv=5, scoring= 'accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('missing_value',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('categorical_value',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('scaler_value',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('fe_selection',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7c6455e27880>)),
                                       ('d_tree', DecisionTreeClassifier())]),
             param_grid={'d_tree__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [31]:
grid.best_score_

0.6391214419383433

In [32]:
grid.best_params_

{'d_tree__max_depth': 2}

In [33]:
import pickle
pickle.dump(pipe,open('pipe.pkl', 'wb'))

In [34]:
pipe= pickle.load(open('pipe.pkl', 'rb'))

In [35]:
#assume user input
test_input2= np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'], dtype= object).reshape(1, 7)

In [36]:
pipe.predict(test_input2)

array([0])